In [1]:
import requests
from sqlalchemy import create_engine
import pandas as pd
import io
import os
from multiprocessing import Pool, cpu_count

# using census.gov tools

## Understanding Geographic Identifiers (GEOIDs)
https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

## Geocoder documentation
https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/census-geocoder.html

### Column info for the geography batch address output
<table>
<tbody><tr><th>Column<br>
</th>
<th>Column Name</th>
<th>Column Description</th>
</tr><tr><td>1</td>
<td>Record ID Number</td>
<td>ID from original address list</td>
</tr><tr><td>2</td>
<td>Input Address</td>
<td>Address from original address list</td>
</tr><tr><td>3</td>
<td>TIGER Address Range Match Indicator</td>
<td>Results indicating whether or not there was a match for the address (Match, tie, no match)<br>
</td>
</tr><tr><td>4</td>
<td>TIGER Match Type</td>
<td>Results indicating if the match is exact or not (Exact, non-exact)<br>
</td>
</tr><tr><td>5</td>
<td>TIGER Output Address</td>
<td>Address the original address matches to<br>
</td>
</tr><tr><td>6</td>
<td>Interpolated Longitude, Latitude</td>
<td>Interpolated longitude and latitude for the address<br>
</td>
</tr><tr><td>7</td>
<td>TIGER Line ID</td>
<td>Unique ID for the edge the address falls on in the MAF/TIGER database<br>
</td>
</tr><tr><td>8</td>
<td>TIGER Line ID Side</td>
<td>Side of the street address in on (L for left and R for right)<br>
</td>
</tr><tr><td>9</td>
<td>State Code</td>
<td>State FIPS Code<br>
</td>
</tr><tr><td>10</td>
<td>County Code</td>
<td>County FIPS Code<br>
</td>
</tr><tr><td>11</td>
<td>Tract Code</td>
<td>Census Tract Code<br>
</td>
</tr><tr><td>12</td>
<td>Block Code</td>
<td>Census Block Code</td>
</tr></tbody>
</table>


### Hierarchy Diagrams
https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

<table cellspacing="0" cellpadding="0" border="1">
<tbody><tr><th><p><strong>Area Type</strong></p>
</th>
<th><p><strong>GEOID Structure</strong></p>
</th>
<th><p><strong>Number of Digits</strong></p>
</th>
<th><p><strong>Example Geographic Area</strong></p>
</th>
<th><p><strong>Example GEOID</strong></p>
</th>
</tr><tr><td><p>State</p>
</td>
<td><p>STATE</p>
</td>
<td><p>2</p>
</td>
<td><p>Texas</p>
</td>
<td><p>48</p>
</td>
</tr><tr><td><p>County</p>
</td>
<td><p>STATE+COUNTY</p>
</td>
<td><p>2+3=5</p>
</td>
<td><p>Harris County, TX</p>
</td>
<td><p>48201</p>
</td>
</tr><tr><td><p>County Subdivision</p>
</td>
<td><p>STATE+COUNTY+COUSUB</p>
</td>
<td><p>2+3+5=10</p>
</td>
<td><p>Pasadena CCD, Harris County, TX</p>
</td>
<td><p>4820192975</p>
</td>
</tr><tr><td><p>Places</p>
</td>
<td><p>STATE+PLACE</p>
</td>
<td><p>2+5=7</p>
</td>
<td><p>Houston, TX</p>
</td>
<td><p>4835000</p>
</td>
</tr><tr><td><p>Census Tract</p>
</td>
<td><p>STATE+COUNTY+TRACT</p>
</td>
<td><p>2+3+6=11</p>
</td>
<td><p>Census Tract 2231 in Harris County, TX</p>
</td>
<td><p>48201223100</p>
</td>
</tr><tr><td><p>Block Group</p>
</td>
<td><p>STATE+COUNTY+TRACT+BLOCK GROUP</p>
</td>
<td><p>2+3+6+1=12</p>
</td>
<td><p>Block Group 1 in Census Tract 2231 in Harris County, TX</p>
</td>
<td><p>482012231001</p>
</td>
</tr><tr><td><p>Block*</p>
</td>
<td><p>STATE+COUNTY+TRACT+BLOCK</p>
</td>
<td><p>2+3+6+4=15 (Note – some blocks also contain a one character suffix (A, B, C, ect.)</p>
</td>
<td><p>Block 1050 in Census Tract 2231 in Harris County, TX</p>
</td>
<td><p>482012231001050</p>
</td>
</tr><tr><td><p>Congressional District (113th Congress)</p>
</td>
<td><p>STATE+CD</p>
</td>
<td><p>2+2=4</p>
</td>
<td><p>Connecticut District 2</p>
</td>
<td><p>0902</p>
</td>
</tr><tr><td><p>State Legislative District (Upper Chamber)</p>
</td>
<td><p>STATE+SLDU</p>
</td>
<td><p>2+3=5</p>
</td>
<td><p>Connecticut State Senate District 33</p>
</td>
<td><p>09033</p>
</td>
</tr><tr><td><p>State Legislative District (Lower Chamber)</p>
</td>
<td><p>STATE+SLDL</p>
</td>
<td><p>2+3=5</p>
</td>
<td><p>Connecticut State House District 147</p>
</td>
<td><p>09147</p>
</td>
</tr><tr><td><p>ZCTA **</p>
</td>
<td><p>ZCTA</p>
</td>
<td><p>5</p>
</td>
<td><p>Suitland, MD ZCTA</p>
</td>
<td><p>20746</p>
</td>
</tr><tr><td colspan="5"><p>* The block group code is not included in the census block GEOID code because the first digit of a census block code represents the block group code.</p>
</td>
</tr><tr><td colspan="5"><p>** ZIP Code Tabulation Areas (ZCTAs) are generalized areal representations of United States Postal Service (USPS) ZIP Code service areas.</p>
</td>
</tr></tbody></table>

### Note

Address data in the file is formatted like this
(leave out the first line if it defines the column names):

`id,Street address,City,State,Zip`

or, this for example:

`481038546029,123 Fake Ln,Cincinnati,OH,45236`

### NOTE:
the sqlite database has been pre-created with 2 tables:

    * patron_address_data
    * geocoding_output
    
**also note** the table for geocoding_output has been set up to replace updated values for address data on conflicting primary keys

```sql
CREATE TABLE "patron_address_data" (
	"record_num"	INTEGER,
	"id"	INTEGER,
	"creation_date"	TEXT,
	"record_last_updated_date"	TEXT,
	"ptype_code"	INTEGER,
	"home_library_code"	TEXT,
	"expiration_date_gmt"	TEXT,
	"birth_date_gmt"	TEXT,
	"checkout_total"	INTEGER,
	"renewal_total"	INTEGER,
	"activity_gmt"	TEXT,
	"addr1"	TEXT,
	"addr2"	TEXT,
	"city"	TEXT,
	"state"	TEXT,
	"postal_code"	TEXT
);

--

CREATE TABLE "geocoding_output" (
	"id"	BIGINT,
	"address"	TEXT,
	"outcome"	TEXT,
	"confidence"	TEXT,
	"census_address"	TEXT,
	"cord"	TEXT,
	"tiger"	BIGINT,
	"side"	TEXT,
	"state_fips_code"	TEXT,
	"county_fips_code"	TEXT,
	"census_tract_code"	TEXT,
	"census_block_code"	TEXT,
	PRIMARY KEY("id") ON CONFLICT REPLACE
);

```

### the sierra query to fill the sqlite table patron_address_data is below:
```sql
select
r.record_num,
r.id,
date(r.creation_date_gmt) as creation_date,
date(r.record_last_updated_gmt) as record_last_updated_date,
p.ptype_code,
p.home_library_code,
p.expiration_date_gmt,
p.birth_date_gmt,
p.checkout_total,
p.renewal_total,
p.activity_gmt,
(
	SELECT
	a.addr1
	from
	sierra_view.patron_record_address as a
	where
	a.patron_record_id = r.id

	order by
	display_order,
	patron_record_address_type_id

	limit 1
) as addr1,
(
	SELECT
	a.addr2
	from
	sierra_view.patron_record_address as a
	where
	a.patron_record_id = r.id

	order by
	display_order,
	patron_record_address_type_id

	limit 1
) as addr2,

(
	SELECT
	a.city
	from
	sierra_view.patron_record_address as a
	where
	a.patron_record_id = r.id

	order by
	display_order,
	patron_record_address_type_id

	limit 1
) as city,
(
	SELECT
	a.region
	from
	sierra_view.patron_record_address as a
	where
	a.patron_record_id = r.id

	order by
	display_order,
	patron_record_address_type_id

	limit 1
) as state,
(
	SELECT
	a.postal_code
	from
	sierra_view.patron_record_address as a
	where
	a.patron_record_id = r.id

	order by
	display_order,
	patron_record_address_type_id

	limit 1
) as postal_code

from
sierra_view.record_metadata as r

JOIN
sierra_view.patron_record as p
ON
  p.record_id = r.id

where
r.record_type_code = 'p'
and r.campus_code = ''
and r.deletion_date_gmt is null
```

In [2]:
# this is the name of our local database that has the address data pre-populated as described above
connection_uri = 'sqlite:///patron_address.db'

# use sqlalchemy to create our database connection
engine = create_engine(connection_uri)

In [3]:
# get the number of rows of addresses in our address table 
num_rows = engine.execute('select count(*) as count from patron_address_data').fetchone()[0]
print('num_rows: {}'.format(num_rows))

num_rows: 599877


In [4]:
# this is the sql for the local database to assemble the csv data
# NOTE that we'll supply the offset value in the loop below
sql = '''
SELECT
id,
addr1 || coalesce(' ' || addr2, '') as 'Street address',
coalesce(city, ' ') as City,
coalesce(state, ' ') as State,
coalesce(postal_code, ' ')  as Zip

FROM
patron_address_data

ORDER BY
id DESC

LIMIT 10000 OFFSET ?
'''

In [5]:
def gen_csv(i):
    with engine.connect() as conn:
        df = pd.read_sql(sql, conn, params=[i, ], )
        df.to_csv('./csv_output/geocoding_input_' + str(i) + '.csv', header=False, index=False )

In [6]:
p = Pool(cpu_count())
output = p.map( gen_csv, (i for i in range(0, num_rows, 10000)) )

In [7]:
# just double check that the number of output files matches the output length
!ls csv_output/ | wc -l
print(len(output))

60
60


In [8]:
def process_csv(csv_file):
    r = requests.post(
            # use locations gives different results ... 
            #url='https://geocoding.geo.census.gov/geocoder/locations/addressbatch',
            url='https://geocoding.geo.census.gov/geocoder/geographies/addressbatch',
            
            files={
                'addressFile': ('input.csv', open(csv_file, 'rb'), 'text/csv'),
            },
            data={  
                'benchmark': 4,
                'vintage': 4,
            }
        )
    
    df_output = pd.read_csv(io.StringIO(r.content.decode('iso-8859-1')),
                            header=None,
                            names=('id', 'address', 'outcome', 'confidence', 'census_address', 'cord', 'tiger', 'side', 'state_fips_code', 'county_fips_code', 'census_tract_code', 'census_block_code'),
                            # note we have to use the pandas integer data type, since there are possible null values
                            dtype={
                                'tiger': pd.Int64Dtype(),
                                'state_fips_code': str,
                                'county_fips_code': str,
                                'census_tract_code': str,
                                'census_block_code': str
                            }
    )
        
    # output the contents to a csv file
    file_name = 'csv_output/geocoding_output_' + csv_file[27:]
    df_output.to_csv(file_name, header=True, index=False )
        
    # also write output back to sqlite database (should already be created, so append to it)
    # NOTE: I don't know if we want to empty the table before doing this, just to make sure we only get fresh data
    # https://github.com/pandas-dev/pandas/issues/15988
    with engine.connect() as conn:
        df_output.to_sql('geocoding_output', conn, index=False, if_exists='append')
    
    try:
        file = open(csv_file)
        return('csv_output/geocoding_output_' + csv_file[27:])
    except:
        return(False)


In [ ]:
files = [f for f in os.listdir('csv_output') if f.endswith('.csv')]

p = Pool(cpu_count())
output = p.map(process_csv, ['csv_output/' + f for f in files])

In [ ]:
output